In [1]:
from dataloader import load_raw, create_datasets
from measures import quantile_loss

import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import json
from datetime import datetime

/Users/stefano.damato/switchdrive/local_packages/gluon-ts/src/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
quantiles = [0.25, 0.5, 0.8, 0.9, 0.95, 0.99]
quantiles_str = ", ".join([str(q) for q in quantiles])
quantiles_str = f"c({quantiles_str})"

importr('forecast')
importr('smooth')

robjects.r(f"""
    quantiles <- {quantiles_str}
    iETS <- function(z, h, levels=quantiles) {{
        set.seed(0)
        model <- smooth::adam(z, model="MNN", occurrence="auto")
        pred <- forecast::forecast(model, h=h, interval='simulated', nsim=50000, level=levels, scenarios=T, side='upper')
        return(pred)
    }}
""")

In [3]:
dataset_name = "OnlineRetail"
dt = datetime.now().strftime("%Y-%m-%d-%H:%M:%S:%f")

data_raw, data_info = load_raw(dataset_name=dataset_name, datasets_folder_path=os.path.join("..","data"))
datasets = create_datasets(data_raw, data_info)

#### iETS

In [4]:
quantile_forecasts = np.empty(shape=(len(datasets['test']), data_info['h'], len(quantiles)))
actuals = np.zeros(shape=(len(datasets['test']), data_info['h']))
for i in tqdm(range(len(datasets['test']))):
    serie_train = datasets['valid'][i]['target']
    serie_test = datasets['test'][i]['target'][-data_info['h']:]
    quantile_forecasts[i] =  np.array(robjects.r['iETS'](robjects.vectors.FloatVector(serie_train), data_info['h']).rx2('upper'))
    actuals[i] = serie_test

quantile_forecasts.shape # (N x h x Q)

  4%|▍         | 96/2489 [00:54<19:26,  2.05it/s]R[write to console]: Warning:
R[write to console]:  Something went wrong in the estimation of the model and NaNs were produced. If this is a mixed model, consider using the pure ones instead.

  5%|▍         | 119/2489 [01:07<22:37,  1.75it/s]R[write to console]: Warning:
R[write to console]:  Something went wrong in the estimation of the model and NaNs were produced. If this is a mixed model, consider using the pure ones instead.

 11%|█         | 269/2489 [02:23<18:29,  2.00it/s]R[write to console]: Warning:
R[write to console]:  Something went wrong in the estimation of the model and NaNs were produced. If this is a mixed model, consider using the pure ones instead.

 12%|█▏        | 288/2489 [02:32<17:39,  2.08it/s]R[write to console]: Warning:
R[write to console]:  Something went wrong in the estimation of the model and NaNs were produced. If this is a mixed model, consider using the pure ones instead.

 12%|█▏        | 305/2489 [02

(2489, 28, 6)

In [5]:
model_folder_path = os.path.join("..","trained_models", "iETS__"+dataset_name+"__"+dt)
if not os.path.exists(path=model_folder_path):
    os.makedirs(model_folder_path)
    np.save(os.path.join(model_folder_path,"qforecasts.npy"), quantile_forecasts)
    np.save(os.path.join(model_folder_path,"actuals.npy"), actuals)
    np.save(os.path.join(model_folder_path,"q.npy"), quantiles)
    metrics = {
        'quantile_loss' : quantile_loss(actuals, quantile_forecasts, quantiles)
    }
    json.dump(metrics, open(os.path.join(model_folder_path,"metrics.json"), "w"))

#### Zero-forecast

In [6]:
model_folder_path = os.path.join("..","trained_models", "ZeroForecast__"+dataset_name+"__"+dt)
if not os.path.exists(path=model_folder_path):
    os.makedirs(model_folder_path)
    np.save(os.path.join(model_folder_path,"qforecasts.npy"), np.zeros_like(quantile_forecasts))
    np.save(os.path.join(model_folder_path,"actuals.npy"), actuals)
    np.save(os.path.join(model_folder_path,"q.npy"), quantiles)
    metrics = {
        'quantile_loss' : quantile_loss(actuals, np.zeros_like(quantile_forecasts), quantiles)
    }
    json.dump(metrics, open(os.path.join(model_folder_path,"metrics.json"), "w"))

#### NaiveW-forecast

In [7]:
quantile_forecasts_naive = np.empty(shape=(len(datasets['test']), data_info['h'], len(quantiles)))
actuals = np.zeros(shape=(len(datasets['test']), data_info['h']))
for i in tqdm(range(len(datasets['test']))):
    serie_train = datasets['valid'][i]['target']
    serie_test = datasets['test'][i]['target'][-data_info['h']:]
    quantile_forecasts_naive[i] = np.round(np.mean(serie_train[-2*data_info['h']:]))
    actuals[i] = serie_test

quantile_forecasts_naive.shape # (N x h x Q)

100%|██████████| 2489/2489 [00:00<00:00, 6228.22it/s]


(2489, 28, 6)

In [8]:
model_folder_path = os.path.join("..","trained_models", "NaiveW__"+dataset_name+"__"+dt)
if not os.path.exists(path=model_folder_path):
    os.makedirs(model_folder_path)
    np.save(os.path.join(model_folder_path,"qforecasts.npy"), quantile_forecasts_naive)
    np.save(os.path.join(model_folder_path,"actuals.npy"), actuals)
    np.save(os.path.join(model_folder_path,"q.npy"), quantiles)
    metrics = {
        'quantile_loss' : quantile_loss(actuals, quantile_forecasts_naive, quantiles)
    }
    json.dump(metrics, open(os.path.join(model_folder_path,"metrics.json"), "w"))

#### EmpiricalQuantiles-forecast ####

In [9]:
quantile_forecasts_eq = np.empty(shape=(len(datasets['test']), data_info['h'], len(quantiles)))
actuals = np.zeros(shape=(len(datasets['test']), data_info['h']))
for i in tqdm(range(len(datasets['test']))):
    serie_train = datasets['valid'][i]['target']
    serie_test = datasets['test'][i]['target'][-data_info['h']:]
    quantile_forecasts_eq[i] = np.tile(np.quantile(serie_train, quantiles), (data_info['h'], 1))
    actuals[i] = serie_test

quantile_forecasts_eq.shape # (N x h x Q)

100%|██████████| 2489/2489 [00:00<00:00, 5007.28it/s]


(2489, 28, 6)

In [10]:
model_folder_path = os.path.join("..","trained_models", "EmpQ__"+dataset_name+"__"+dt)
if not os.path.exists(path=model_folder_path):
    os.makedirs(model_folder_path)
    np.save(os.path.join(model_folder_path,"qforecasts.npy"), quantile_forecasts_eq)
    np.save(os.path.join(model_folder_path,"actuals.npy"), actuals)
    np.save(os.path.join(model_folder_path,"q.npy"), quantiles)
    metrics = {
        'quantile_loss' : quantile_loss(actuals, quantile_forecasts_eq, quantiles)
    }
    json.dump(metrics, open(os.path.join(model_folder_path,"metrics.json"), "w"))